# EDA

Display the number of images per diffusion model

In [4]:
import os
from collections import defaultdict

root_dir = "../data/art/fake"
valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".webp"}

class_prefixes = ["AI_DiffusionDB", "AI_LD", "AI_SD"]

image_counts = defaultdict(int)


for subfolder in os.listdir(root_dir):
    subfolder_path = os.path.join(root_dir, subfolder)
    if not os.path.isdir(subfolder_path):
        continue


    matched_class = next(
        (prefix for prefix in class_prefixes if subfolder.startswith(prefix)), None)
    if matched_class:

        for file in os.listdir(subfolder_path):
            if os.path.splitext(file)[1].lower() in valid_extensions:
                image_counts[matched_class] += 1


for cls in class_prefixes:
    print(f"{cls}: {image_counts[cls]} images")

AI_DiffusionDB: 21292 images
AI_LD: 31922 images
AI_SD: 6251 images
